In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
from scipy.interpolate import make_interp_spline, BSpline
from cycler import cycler
%matplotlib notebook

In [5]:
df = pd.read_csv('dataset.csv')

In [6]:
df.head()

,ball,probability,outcome,runs,wickets,run_rate,to_win
0,1,58.15,0,0,0,0.0,NaN
1,2,56.99,0,0,0,0.0,NaN
2,3,58.86,1,1,0,2.0,NaN
3,4,61.99,1,2,0,3.0,NaN
4,5,56.92,1,3,0,3.6,NaN


In [4]:
ball = df.ball
prob = df.probability
outcomes = df.outcome
runs = df.runs
wickets = df.wickets
run_rate = df.run_rate
to_win = df.to_win

In [5]:
## Initialization

# Define x and y
x = ball
y = prob

## If you want your curve to be smoothened,
# x = np.linspace(ball.min(), ball.max(), 200)
# spl = make_interp_spline(ball, prob, k=3)  # type: BSpline
# power_smooth = spl(x)
# y = power_smooth

In [67]:
# Actual Curve (best if you have data for each ball)

# Resets the rcParams before running each time
plt.rcParams.update(plt.rcParamsDefault)

plt.rcParams.update({
    "axes.prop_cycle": cycler(color=["#FEE715FF", "#e94cdc", "0.7"]),
    "lines.linewidth": 3,
    "axes.titlecolor": "white",
    "axes.facecolor": "#101820FF",
    "axes.edgecolor": "white",
    "axes.labelcolor": "white",
    "xtick.color": "white",
    "ytick.color": "white",
    "figure.facecolor": "#101820FF",
    "savefig.facecolor": "#101820FF"
})

# Creating the figure
fig, ax = plt.subplots()
line, = ax.plot(x, y)

# Customization
font = {'fontname':'Oswald'}
axis_font = {'fontname':'Oswald Regular'}
subtitle_font = {'fontname':'Tw Cen MT Condensed'}
fig.subplots_adjust(top=0.85)
fig.suptitle("Sri Lanka's Win Probability Variation vs Australia ( T20I - SL vs AUS on $\mathregular{11^{th}}$ June 2022 ) \n Classifier: Logistic Regression", fontsize=15, color='white', y=0.96, **font)
plt.xlabel('Ball', fontsize = 13, **axis_font, labelpad = 20)
plt.ylabel('Win Probability (%)', fontsize = 13, **axis_font, labelpad = 20)
fig.set_size_inches(16, 9)
fig.set_tight_layout({"pad": 7})
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xticks(np.arange(6, 246, 6),**axis_font)
plt.yticks(np.arange(10, 110, 10),**axis_font)
ax.set_ylim([0, 100])
line.axes.axis([0, 246, 0, 110])
plt.axvline(x=120, color='white', linestyle='--', label = 'End of First Innings', linewidth=1.5)  
plt.axhline(y=50, color='white', linestyle='--', label = 'End of First Innings', linewidth=1.5)  
plt.figtext(0.99, 0.01, 'predictors.shinyapps.io/wpsg/', horizontalalignment='right', color = 'white', **axis_font)
result = plt.figtext(0.25,0.95, "", bbox={'facecolor':'w', 'alpha':0.8, 'pad':1.2, 'boxstyle':'round', 'edgecolor': '#101820FF'}, transform=ax.transAxes, ha="center", color = 'white', **font)
t1 = ax.text(0,1.05, "", transform = ax.transAxes, ha="left", color='yellow', fontsize=17, **subtitle_font)
t2 = ax.text(0.5,1.05, "", transform = ax.transAxes, ha="center", color='yellow', fontsize=17, **subtitle_font)
t3 = ax.text(1,1.05, "", transform = ax.transAxes, ha="right", color='yellow', fontsize=17, **subtitle_font)

def update(num, x, y, line):
    ball_number = int(x[num])
    innings = 1 if ball_number < 121 else 2
#     plt.title("\nSL Win Probability: " + '{:.2f}'.format(round(y[num], 2)) + "% \n", fontsize=13, color='yellow', y=0.91, **font, loc='left', pad=45)
#     plt.title("\nInnings: " + str(innings) + " | Over: " + get_over_number(ball_number) + " (Ball: " + str(ball_number) + ") \n", fontsize=17, color='yellow', y=0.91, **subtitle_font, pad=40)
#     plt.title("\nAUS Win Probability: " + '{:.2f}'.format(round(100-y[num], 2)) + "% \n", fontsize=17, color='yellow', y=0.91, **subtitle_font, loc='right', pad=45)    
    line.set_data(x[:num], y[:num]) 
    t1.set_text("\nSL Win Probability: " + '{:.2f}'.format(round(y[num], 2)) + "%\n")
    t2.set_text("\nInnings: " + str(innings) + "| Bat: " + get_batting_team(ball_number) + " | Score: " + get_text_styled_score(runs[num], wickets[num])
                + " | Overs: " + get_over_number(ball_number) + " | Run Rate: " + '{:.2f}'.format(run_rate[num]) + get_to_win_str(ball_number, to_win[num]) + "\n")
    t3.set_text("\nAUS Win Probability: " + '{:.2f}'.format(round(100-y[num], 2)) + "%\n")
    result.set_text(str(outcomes[num]))
    result.set_backgroundcolor(get_color(outcomes[num]))
    result.set_position(get_outcome_position(ball_number))
    return line,t1,t2,t3,

def get_batting_team(ball):
    if ball < 121:
        return "AUS"
    else:
        return "SL"

def get_over_number(ball):
    if ball < 121:
        over_str = f'{str(ball//6):>2}' + "." + str(ball%6) + " (Balls: " + f'{str(ball):>3}' + ")"
        return over_str
    else:
        b = ball - 120
        over_str = f'{str(b//6):>2}' + "." + str(b%6) + " (Balls: " + f'{str(ball):>3}' + ")"
        return over_str
    
def get_color(outcome):
    if outcome == '0':
        return '#433E47'
    elif outcome == '1':
        return '#35DF91'
    elif outcome == '2':
        return '#FAC762'
    elif outcome == '3':
        return '#FAC762'
    elif outcome == '4':
        return '#009944'
    elif outcome == '5':
        return '#009944'
    elif outcome == '6':
        return '#7A41D8'
    else:
        return '#E44242'

def get_outcome_position(ball):
    if ball < 121:
        return (0.25, 0.95)
    else:
        return (0.75, 0.95)
    
def get_text_styled_score(score, wickets):
    return str(score).rjust(3) + " / " + str(wickets)

def get_to_win_str(ball, to_win):
    if ball > 120:
        return " | To Win: " + str(int(to_win)).rjust(3)
    else:
        return ""

In [70]:
ani = animation.FuncAnimation(fig, update, frames = len(x), fargs=[x, y, line],
                              interval=200, blit=True)

metadata = dict(title="Sri Lanka's T20I win Probability Variation vs Australia (11th June 2022)", artist='Pranath Pussella',
                comment=':)')
dpi = 300
writer = animation.writers['ffmpeg'](fps=10, bitrate = 30000)
ani.save('test.mp4', writer=writer, dpi=dpi)
# plt.show()

In [ ]:
# A basic animated plot without any styling

# x = ball
# y = prob

# fig, ax = plt.subplots()
# line, = ax.plot(x, y, color='k')


# def update(num, x, y, line):
#     plt.xticks(np.arange(12, 250, 12))
#     line.set_data(x[:num], y[:num])
#     line.axes.axis([0, 250, 0, 100])
#     return line,

# ani = animation.FuncAnimation(fig, update, len(x), fargs=[x, y, line],
#                               interval=25, blit=False)
# # ani.save('test.mp4')
# plt.show()

In [ ]:
# A nice little code snippet for getting the available list of font families in the system

# import matplotlib.font_manager
# from IPython.core.display import HTML

# def make_html(fontname):
#     return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

# code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

# HTML("<div style='column-count: 2;'>{}</div>".format(code))

In [29]:

### ARCHIVED

# # Smoothened Curve (does not give accurate results) - Initial non-refactored version using smoothened curve
# plt.rcParams.update(plt.rcParamsDefault)

# plt.rcParams.update({
#     "lines.color": "white",
#     "patch.edgecolor": "white",
#     "text.color": "black",
#     "axes.prop_cycle": cycler(color=["#FEE715FF", "#e94cdc", "0.7"]),
# #     "axes.prop_cycle": cycler(color=["green", "#e94cdc", "0.7"]),
#     "lines.linewidth": 3,
#     "axes.titlecolor": "white",
#     "axes.facecolor": "#101820FF",
#     "axes.edgecolor": "white",
#     "axes.labelcolor": "white",
#     "axes.titlepad": 10.0,
#     "xtick.color": "white",
#     "ytick.color": "white",
#     "grid.color": "black",
#     "figure.facecolor": "#101820FF",
#     "figure.edgecolor": "black",
#     "savefig.facecolor": "#101820FF",
#     "savefig.edgecolor": "black"
# })


# # Generate x and y
# x = ball
# y = prob
# xnew = np.linspace(x.min(), x.max(), 200)
# spl = make_interp_spline(x, y, k=3)  # type: BSpline
# power_smooth = spl(xnew)

# # Generate Title

# fig, ax = plt.subplots()
# line, = ax.plot(xnew, power_smooth)

# fig.subplots_adjust(top=0.85)

# font = {'fontname':'Oswald'}
# axis_font = {'fontname':'Oswald Regular'}
# fig.suptitle("Sri Lanka's Win Probability Variation vs Australia \n (T20I - SL vs AUS on $\mathregular{11^{th}}$ June 2022)", fontsize=15, color='white', y=0.96, **font)
# plt.xlabel('Ball', labelpad = 20, fontsize = 13, **axis_font)
# plt.ylabel('Win Probability (%)', labelpad = 20, fontsize = 13, **axis_font)
# fig.set_size_inches(16, 9)
# fig.set_tight_layout({"pad": 7})
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)
# plt.xticks(np.arange(6, 246, 6),**axis_font)
# plt.yticks(np.arange(10, 110, 10),**axis_font)
# line.axes.axis([0, 246, 0, 110])
# plt.axvline(x=120, color='white', linestyle='--', label = 'End of First Innings', linewidth=1.5)  

# def update(num, x, y, line):
#     ball_number = int(x[num])
#     innings = 1 if ball_number < 121 else 2
#     plt.title("SL Win Probability: " + str(round(y[num],2)) + "%", fontsize=13, color='yellow', y=0.92, **font, loc='left', pad=45)
#     plt.title("Innings: " + str(innings) + " | Over: " + get_over_number(ball_number) + " (Ball: " + str(ball_number) + ")", fontsize=13, color='yellow', y=0.92, **font, pad=40)
#     plt.title("AUS Win Probability: " + str(round((100-y[num]),2)) + "%", fontsize=13, color='yellow', y=0.92, **font, loc='right', pad=45)    
#     line.set_data(x[:num], power_smooth[:num]) 
#     return line,

# def get_over_number(ball):
#     if ball < 121:
#         return str(ball//6) + "." + str(ball%6)
#     else:
#         b = ball - 120
#         return str(b//6) + "." + str(b%6)

# ani = animation.FuncAnimation(fig, update, frames = len(xnew), fargs=[xnew, power_smooth, line],
#                               interval=200, blit=True)

# metadata = dict(title="Sri Lanka's T20I win Probability Variation vs Australia (11th June 2022)", artist='Pranath Pussella',
#                 comment=':)')
# dpi = 300
# writer = animation.writers['ffmpeg'](fps=10, bitrate = 500)
# ani.save('test.mp4', writer=writer, dpi=dpi)
# # plt.show()